# Website Log Data Cleaning and Sessionization

## Objective
The goal of this task is to clean raw website log data and identify user sessions
based on a 30-minute inactivity threshold. Bot and crawler traffic is also filtered
to retain only genuine user activity.

## Dataset Source:
Public Apache web server access log dataset obtained from an open GitHub repository.
The dataset contains real website traffic including human users and bots.


In [2]:
import pandas as pd
import re

log_path = "../data/access.txt"
logs = []

pattern = re.compile(
    r'(\d+\.\d+\.\d+\.\d+).*?\[(.*?)\]\s+"(\w+)\s+(.*?)\s+HTTP.*?"\s+\d+\s+\S+\s+".*?"\s+"(.*?)"'
)

with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        match = pattern.search(line)
        if match:
            logs.append(match.groups())

df = pd.DataFrame(
    logs,
    columns=["ip_address", "timestamp", "method", "url", "user_agent"]
)

df.head()


,ip_address,timestamp,method,url,user_agent
0,83.149.9.216,17/May/2015:10:05:03 +0000,GET,/presentations/logstash-monitorama-2013/images...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
1,83.149.9.216,17/May/2015:10:05:43 +0000,GET,/presentations/logstash-monitorama-2013/images...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
2,83.149.9.216,17/May/2015:10:05:47 +0000,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
3,83.149.9.216,17/May/2015:10:05:12 +0000,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
4,83.149.9.216,17/May/2015:10:05:07 +0000,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...


### Convert timestamp to datetime

In [4]:
df["timestamp"] = pd.to_datetime(
    df["timestamp"],
    format="%d/%b/%Y:%H:%M:%S %z"
)

df.head()

,ip_address,timestamp,method,url,user_agent
0,83.149.9.216,2015-05-17 10:05:03+00:00,GET,/presentations/logstash-monitorama-2013/images...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
1,83.149.9.216,2015-05-17 10:05:43+00:00,GET,/presentations/logstash-monitorama-2013/images...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
2,83.149.9.216,2015-05-17 10:05:47+00:00,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
3,83.149.9.216,2015-05-17 10:05:12+00:00,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
4,83.149.9.216,2015-05-17 10:05:07+00:00,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...


### Sort data by user and time

In [7]:
df = df.sort_values(by=["ip_address", "timestamp"])
df.head()

,ip_address,timestamp,method,url,user_agent
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...


### Calculate time difference between requests

In [10]:
df["time_diff"] = df.groupby("ip_address")["timestamp"].diff()
df.head()

,ip_address,timestamp,method,url,user_agent,time_diff
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03


### Apply 30-minute session rule

In [13]:
session_timeout = pd.Timedelta(minutes=30)

df["new_session"] = (
    (df["time_diff"].isna()) | (df["time_diff"] > session_timeout)
)
df.head()

,ip_address,timestamp,method,url,user_agent,time_diff,new_session
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT,True
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20,False
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09,False
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07,False
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03,False


### Create session_id

In [16]:
df["session_id"] = (
    df.groupby("ip_address")["new_session"].cumsum()
)

df.head()

,ip_address,timestamp,method,url,user_agent,time_diff,new_session,session_id
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT,True,1
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20,False,1
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09,False,1
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07,False,1
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03,False,1


### Make session_id more readable

In [19]:
df["session_id"] = (
    df["ip_address"] + "_S" + df["session_id"].astype(str)
)

df.head()

,ip_address,timestamp,method,url,user_agent,time_diff,new_session,session_id
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT,True,1.22.35.226_S1
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20,False,1.22.35.226_S1
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09,False,1.22.35.226_S1
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07,False,1.22.35.226_S1
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03,False,1.22.35.226_S1


### Sessionization Logic
User sessions were identified by grouping log entries by IP address and sorting them
by timestamp. A new session was created whenever the time difference between consecutive
requests exceeded 30 minutes.


### Identify bot / crawler traffic

In [23]:
bot_keywords = [
    "bot", "spider", "crawl", "slurp", "archiver", "crawler"
]

df["is_bot"] = df["user_agent"].str.lower().str.contains(
    "|".join(bot_keywords),
    na=False
)

df[["user_agent", "is_bot"]].head()

,user_agent,is_bot
5857,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False
5862,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False
5858,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False
5863,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False
5855,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False


### Remove bot traffic

In [26]:
df_clean = df[~df["is_bot"]].copy()
df_clean.head()

,ip_address,timestamp,method,url,user_agent,time_diff,new_session,session_id,is_bot
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT,True,1.22.35.226_S1,False
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20,False,1.22.35.226_S1,False
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09,False,1.22.35.226_S1,False
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07,False,1.22.35.226_S1,False
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03,False,1.22.35.226_S1,False


In [28]:
print("Original rows:", len(df))
print("After bot removal:", len(df_clean))

Original rows: 9999
After bot removal: 8602


### Create final session-level summary

In [31]:
session_summary = (
    df_clean
    .groupby(["session_id", "ip_address"])
    .agg(
        session_start=("timestamp", "min"),
        session_end=("timestamp", "max"),
        page_views=("url", "count"),
        unique_pages=("url", "nunique")
    )
    .reset_index()
)

session_summary.head()

,session_id,ip_address,session_start,session_end,page_views,unique_pages
0,1.22.35.226_S1,1.22.35.226,2015-05-19 11:05:07+00:00,2015-05-19 11:05:49+00:00,6,6
1,100.2.4.116_S1,100.2.4.116,2015-05-18 21:05:22+00:00,2015-05-18 21:05:53+00:00,2,2
2,100.2.4.116_S2,100.2.4.116,2015-05-18 23:05:16+00:00,2015-05-18 23:05:23+00:00,2,1
3,100.2.4.116_S3,100.2.4.116,2015-05-19 04:05:22+00:00,2015-05-19 04:05:31+00:00,2,2
4,101.119.18.35_S1,101.119.18.35,2015-05-19 16:05:00+00:00,2015-05-19 16:05:56+00:00,33,33


### Save final output to CSV

In [37]:
session_summary.to_csv(
    "../data/sessionized_logs.csv",
    index=False
)

### Bot Filtering
Bot and crawler traffic was identified using keywords such as 'bot', 'spider',
and 'crawler' in the user-agent field. These records were excluded to ensure
session analysis reflects genuine user behavior.

### Final Output
The final dataset contains session-level summaries including session duration
and page interaction metrics, suitable for downstream user behavior analysis.
